# Naive RAG 实现



In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("DASHSCOPE_API_KEY")

if api_key: print("🔑 API key loaded")

🔑 API key loaded


# Indexing 索引

核心步骤：

1. 文档加载 Document Loading
2. 文本分块 Text Splitting
3. 向量化 Vectorization
4. 存储 Storage

## 文档加载

In [19]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
import os

# 加载 pdf 文件
pdf_directory = "../doc/金融新闻pdf/"
pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

all_docs = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_docs.extend(docs)

print(f"加载的文档数量为: {len(all_docs)}")
# print(f"{all_docs[0].page_content[:200]}\n")
# print(all_docs[0].metadata)

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)


加载的文档数量为: 17


## 文本分块

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(all_docs)

print(f"分割的块有：{len(all_splits)}个")

分割的块有：26个


## 向量化数据

In [24]:
from langchain_community.embeddings import DashScopeEmbeddings

# 创建嵌入模型
embeddings_model = DashScopeEmbeddings(
    model="text-embedding-v4",
)

## 存储数据

In [27]:
from langchain_community.vectorstores import Chroma

# 创建向量数据库
vectorstore = Chroma.from_documents(
    documents=all_splits,           # 分块后的文档
    embedding=embeddings_model,       # 嵌入模型
    persist_directory="./chroma_db"  # 持久化目录
)

print(f"✅ 向量数据库创建成功！")
print(f"存储了 {len(all_splits)} 个文档块")
print(f"持久化路径: ./chroma_db_")

✅ 向量数据库创建成功！
存储了 26 个文档块
持久化路径: ./chroma_db_
✅ 向量数据库创建成功！
存储了 26 个文档块
持久化路径: ./chroma_db_


# 检索


In [31]:
# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 相似度搜索
    search_kwargs={"k": 5}     # 返回前3个结果
)

# 执行检索
query = "请根据最新的新闻报道，分析美国科技行业目前投资风险性如何，是否存在泡沫。"
docs = retriever.invoke(query)

# 查看结果
for i, doc in enumerate(docs, 1):
    print(f"\n📄 结果 {i}:")
    print(f"内容: {doc.page_content[:200]}...")
    print(f"来源: {doc.metadata.get('source', 'N/A')}")


📄 结果 1:
内容: 美国科技⾏业出现裁员潮 在美股是否存在科技股泡沫的疑问之下，今年以来，多家美国科技巨头掀起裁员⻛暴。据外媒报道，在8⽉初英特尔财报创下56年来最差导致股价暴跌之后，该公司宣布裁员15000⼈。 戴尔正在整顿销售团队包括裁员计划， 并建⽴专注于⼈⼯智能产品和服务的新团队。此次重组可能涉及裁员约12500⼈，占全球员⼯总数的10%。⽹通设备⼤⼚思科⽇前也宣布全球裁员7%，相当于削减⼤约6300个⼯作岗...
来源: ../doc/金融新闻pdf/美国科技行业出现裁员潮.pdf

📄 结果 2:
内容: 美国科技⾏业出现裁员潮 在美股是否存在科技股泡沫的疑问之下，今年以来，多家美国科技巨头掀起裁员⻛暴。据外媒报道，在8⽉初英特尔财报创下56年来最差导致股价暴跌之后，该公司宣布裁员15000⼈。 戴尔正在整顿销售团队包括裁员计划， 并建⽴专注于⼈⼯智能产品和服务的新团队。此次重组可能涉及裁员约12500⼈，占全球员⼯总数的10%。⽹通设备⼤⼚思科⽇前也宣布全球裁员7%，相当于削减⼤约6300个⼯作岗...
来源: ../doc/金融新闻pdf/美国科技行业出现裁员潮.pdf

📄 结果 3:
内容: 但伴随着全球新冠疫情步⼊终结， 美国科技公司竞争加剧。 不少⼤型科技公司意识到精简组织以提⾼利润的必要性。 有分析认为， 基于⼈⼯智能将改变世界的信念， 美国⼤型科技公司的资本⽀出规模相当于历史上规模最⼤、 速度最快的基础设施部署， ⼤型科技公司预计在未来两年内将花费5000亿美元。在如此⼤成本投⼊之下，如盈利远不及预期，可能会导致美国科技公司出现裁员、降薪、回购股票、获利回吐等现象。 责任编辑：...
来源: ../doc/金融新闻pdf/美国科技行业出现裁员潮.pdf

📄 结果 4:
内容: 但伴随着全球新冠疫情步⼊终结， 美国科技公司竞争加剧。 不少⼤型科技公司意识到精简组织以提⾼利润的必要性。 有分析认为， 基于⼈⼯智能将改变世界的信念， 美国⼤型科技公司的资本⽀出规模相当于历史上规模最⼤、 速度最快的基础设施部署， ⼤型科技公司预计在未来两年内将花费5000亿美元。在如此⼤成本投⼊之下，如盈利远不及预期，可能会导致美国科技公司出现裁员、降薪、回购股票、获利回吐等现象。 责任编辑：...
来源: ../doc/金融新闻pdf/美国科技行业出现

# 生成

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_qwq import ChatQwen
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 创建LLM
llm = ChatQwen(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0  # 0 = 更精确, 1 = 更有创意
)

# 定义 prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个AI助手。请根据以下背景信息回答用户的问题。\n\n背景信息：\n{context}"),
    ("human", "{query}"),
])


# 1. 定义格式化函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 2. 定义并行运行的步骤（既保留文档对象，又生成 prompt 需要的字符串）
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)

# 3. 定义生成答案的分支
qa_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) # 这里只把 context 转字符串给 prompt 用
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 合并链：先检索，然后把结果由 assign 加到字典里
final_chain = setup_and_retrieval.assign(answer=qa_chain)

# 5. 提问
query = "请根据最新的新闻报道，分析美国科技行业目前投资风险性如何，是否存在泡沫，给出是或否的具体回答。"
result = final_chain.invoke(query)

# 6. 结果现在是一部字典，包含所有信息
print("问题:", result["query"])
print("回答:", result["answer"])
print("来源:", result["context"]) # 这里就是 source_documents

问题: 请根据最新的新闻报道，分析美国科技行业目前投资风险性如何，是否存在泡沫，给出是或否的具体回答。
回答: **是，存在泡沫。**

根据最新新闻报道和行业趋势分析，美国科技行业目前存在明显的投资泡沫风险，具体依据如下：

1. **大规模裁员暴露前期过度扩张**  
   自2022年以来，美国科技行业连续三年出现大规模裁员（2022年16.5万人，2023年26万人，2024年已超12.4万人），涉及英特尔、戴尔、思科、微软、特斯拉等多家巨头。这一现象的根本原因在于疫情期间的过度招聘——在廉价资金和远程办公需求激增的推动下，科技公司盲目扩编，员工总数最高增长达35%以上（如Meta暴涨60%）。这种非理性扩张反映出当时市场估值脱离实际经营需求，具有典型泡沫特征。

2. **资本支出激增但盈利前景不明**  
   报道指出，美国大型科技公司预计在未来两年投入高达**5000亿美元**用于人工智能等基础设施建设。如此庞大的资本开支并非基于当前收益，而是建立在“人工智能将改变世界”的预期之上。这种以未来愿景为支撑的大规模投资，与互联网泡沫时期的“眼球经济”逻辑相似，存在严重估值虚高风险。

3. **股价表现与基本面脱节**  
   尽管英特尔等公司已出现“56年来最差财报”并导致股价暴跌，但整体科技股仍受市场热捧，尤其AI相关概念股持续获得高溢价。这表明市场情绪仍处于亢奋状态，投资者更关注概念叙事而非盈利能力，进一步加剧了资产价格泡沫化。

4. **历史周期重演：从宽松货币到泡沫破裂**  
   疫情期间全球央行大举降息，催生了元宇宙、加密货币、自动驾驶等一系列投机热潮，大量资本涌入科技领域，形成“低利率驱动”的泡沫环境。如今随着加息周期推进、经济回归常态，泡沫开始出清——裁员正是去杠杆和挤泡沫的过程。

综上所述，美国科技行业在经历了疫情期的非理性繁荣后，正面临结构性调整。尽管人工智能带来长期潜力，但短期内巨额投入与盈利不匹配、企业频繁裁员、估值高企等问题叠加，明确指向**科技股存在泡沫**。

**结论：是，当前美国科技行业存在泡沫。**
来源: [Document(metadata={'creationdate': "D:20241108031850Z00'00'", 'page_label': '1', 'producer': 'macOS 版